In [1]:
import nltk
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from difflib import SequenceMatcher

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
attributes=pd.read_csv('attributes.csv')

In [3]:
### Test if a row misses product_uid also miss the other terms

def all_nan(x):
    return np.array(map(np.isnan, x)).all()

print attributes.loc[np.isnan(attributes['product_uid'])].apply(all_nan, axis=1).all()

attributes=attributes.dropna(how='all')
print attributes.loc[np.isnan(attributes['product_uid'])].shape
attributes['product_uid']=attributes['product_uid'].astype(int)
attributes.head()

True
(0, 3)


,product_uid,name,value
0,100001,Bullet01,Versatile connector for various 90° connection...
1,100001,Bullet02,Stronger than angled nailing or screw fastenin...
2,100001,Bullet03,Help ensure joints are consistently straight a...
3,100001,Bullet04,Dimensions: 3 in. x 3 in. x 1-1/2 in.
4,100001,Bullet05,Made from 12-Gauge steel


In [4]:
lennames = len(set(attributes['name']))
num_names_match = dict(map(lambda x, y: (x, y), set(attributes['name']), range(lennames)))
attributes_hash=attributes.copy()
attributes_hash['name'] = map(lambda x: num_names_match[x], attributes_hash['name'])
attributes_hash.head()

,product_uid,name,value
0,100001,4365,Versatile connector for various 90° connection...
1,100001,4366,Stronger than angled nailing or screw fastenin...
2,100001,4367,Help ensure joints are consistently straight a...
3,100001,4368,Dimensions: 3 in. x 3 in. x 1-1/2 in.
4,100001,4369,Made from 12-Gauge steel


In [5]:
train.head()

,id,product_uid,product_title,search_term,relevance
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67


In [6]:
len(num_names_match)

5410

In [7]:
tmp = train.groupby('product_uid').agg({'product_title':'first'}).reset_index()
tmp['name'] = len(num_names_match)
num_names_match['product_title'] = len(num_names_match)
tmp.columns=['product_uid', 'value', 'name']
tmp.head()
attributes_hash = attributes_hash.append(tmp)

def detect_nan(x):
    if isinstance(x, str):
        return True
    else:
        return not np.isnan(x)

attributes_hash=attributes_hash.loc[map(detect_nan, attributes_hash['value'])]


attributes_hash.head()

,name,product_uid,value
0,4365,100001,Versatile connector for various 90° connection...
1,4366,100001,Stronger than angled nailing or screw fastenin...
2,4367,100001,Help ensure joints are consistently straight a...
3,4368,100001,Dimensions: 3 in. x 3 in. x 1-1/2 in.
4,4369,100001,Made from 12-Gauge steel


In [ ]:
### sanity check: I found the problem that a product_uid can have more than one product title
### I didn't try to take care of this though

mismatch= pd.merge(attributes_hash.loc[attributes_hash['name']==5410], train, 
         how='left', on='product_uid').apply(lambda x: x['value']!=x['product_title'], axis=1)


def myfunc(x):
    return len(set(x))==1

leng = train.groupby('product_uid').agg({'product_title':myfunc}).reset_index()
leng.loc[map(lambda x: not x, leng['product_title'])]
A = set(list(leng.loc[map(lambda x: not x, leng['product_title']), 'product_uid']))
for i in train.loc[train['product_uid']==100540, 'product_title']:
    print i

In [8]:
train_input_output = pd.merge(train, attributes_hash, on='product_uid')[[u'id', u'search_term', u'relevance',u'name', u'value']]
train_input_output.head()

,id,search_term,relevance,name,value
0,2,angle bracket,3.0,4365,Versatile connector for various 90° connection...
1,2,angle bracket,3.0,4366,Stronger than angled nailing or screw fastenin...
2,2,angle bracket,3.0,4367,Help ensure joints are consistently straight a...
3,2,angle bracket,3.0,4368,Dimensions: 3 in. x 3 in. x 1-1/2 in.
4,2,angle bracket,3.0,4369,Made from 12-Gauge steel


In [11]:
import time

start_ = time.time()


def match(a,b):
    if isinstance(a,basestring)==False:
        return 0
    else:
        return SequenceMatcher(None,a,b).ratio()
train_input_output['matching_ratio']=train_input_output.apply(lambda t: match(t['search_term'], t['value']), axis=1) #match(t['search_term'], t['value'])


print time.time()-start_

197.216872931


In [12]:
train_input_output.head()

,id,search_term,relevance,name,value,matching_ratio
0,2,angle bracket,3.0,4365,Versatile connector for various 90° connection...,0.209302
1,2,angle bracket,3.0,4366,Stronger than angled nailing or screw fastenin...,0.272727
2,2,angle bracket,3.0,4367,Help ensure joints are consistently straight a...,0.088235
3,2,angle bracket,3.0,4368,Dimensions: 3 in. x 3 in. x 1-1/2 in.,0.080000
4,2,angle bracket,3.0,4369,Made from 12-Gauge steel,0.378378


In [16]:
n = 1352
match(train_input_output.iloc[n, 1], train_input_output.iloc[n, 4])==train_input_output.iloc[n, 5]

True

In [10]:
test.head()

,id,product_uid,product_title,search_term
0,1,100001,Simpson Strong-Tie 12-Gauge Angle,90 degree bracket
1,4,100001,Simpson Strong-Tie 12-Gauge Angle,metal l brackets
2,5,100001,Simpson Strong-Tie 12-Gauge Angle,simpson sku able
3,6,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong ties
4,7,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong tie hcc668


In [250]:
All=pd.merge(Trans_Attr,train,left_index=True,right_on='product_uid')

In [ ]:
1+1

In [17]:
t = train_input_output.iloc[0,:]
t

id                                                             2
search_term                                        angle bracket
relevance                                                      3
name                                                        4365
value          Versatile connector for various 90° connection...
Name: 0, dtype: object

In [26]:
for i in range(train_input_output.shape[0]):
    t = train_input_output.iloc[i,:]
    try:
        match(t['search_term'], t['value'])
    except Exception as e:
        print i, e
        print '#'*60

2540 'float' object is not iterable
############################################################
2566 'float' object is not iterable
############################################################
2592 'float' object is not iterable
############################################################
11294 'float' object is not iterable
############################################################
11330 'float' object is not iterable
############################################################
11366 'float' object is not iterable
############################################################
18284 'float' object is not iterable
############################################################
18310 'float' object is not iterable
############################################################
18336 'float' object is not iterable
############################################################
19013 'float' object is not iterable
############################################################
19014 'float' object is

KeyboardInterrupt: 

In [34]:
train_input_output.iloc[2540,:]

id                          351
search_term    battery lanterns
relevance                     3
name                       1410
value                       NaN
Name: 2540, dtype: object

In [29]:
train[train['id']==351]

,id,product_uid,product_title,search_term,relevance
99,351,100058,Gama Sonic 40-LED Rechargeable Battery-Powered...,battery lanterns,3.0


In [62]:
def detect_nan(x):
    if isinstance(x, str):
        return True
    else:
        return not np.isnan(x)

attributes_hash.loc[map(detect_nan, attributes_hash['value'])]

,name,product_uid,value
0,4365,100001,Versatile connector for various 90° connection...
1,4366,100001,Stronger than angled nailing or screw fastenin...
2,4367,100001,Help ensure joints are consistently straight a...
3,4368,100001,Dimensions: 3 in. x 3 in. x 1-1/2 in.
4,4369,100001,Made from 12-Gauge steel
5,4370,100001,Galvanized for extra corrosion resistance
6,4371,100001,Install with 10d common nails or #9 x 1-1/2 in...
7,4707,100001,12
8,4494,100001,Galvanized Steel
9,5092,100001,Simpson Strong-Tie


In [57]:
(attributes_hash['value'].loc[map(detect_nan, attributes_hash['value'])]=='I love Python').any()

False